#### Deploy solution

This notebook deploys solution accelerator to the specified workspace.

**What is happening in this notebook?**
 - It downloads the latest set of source files from Github
 - It deploys/updates the Fabric items in the current workspace

In [1]:
%pip install ms-fabric-cli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Import of needed libaries

In [ ]:
import subprocess
import os
import json
from zipfile import ZipFile 
import shutil
import re
import requests
import zipfile
from io import BytesIO
import yaml
import time
import sempy.fabric as fabric

## Download of source & config files
This part downloads all source and config files of AMI needed for the deployment into the ressources of the notebook

In [3]:
def download_folder_as_zip(repo_owner, repo_name, output_zip, branch="main", folder_to_extract="workspace",  remove_folder_prefix = ""):
    # Construct the URL for the GitHub API to download the repository as a zip file
    url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/zipball/{branch}"
    
    # Make a request to the GitHub API
    response = requests.get(url)
    response.raise_for_status()
    
    # Ensure the directory for the output zip file exists
    os.makedirs(os.path.dirname(output_zip), exist_ok=True)
    
    # Create a zip file in memory
    with zipfile.ZipFile(BytesIO(response.content)) as zipf:
        with zipfile.ZipFile(output_zip, 'w') as output_zipf:
            for file_info in zipf.infolist():
                parts = file_info.filename.split('/')
                if  re.sub(r'^.*?/', '/', file_info.filename).startswith(folder_to_extract): 
                    # Extract only the specified folder
                    file_data = zipf.read(file_info.filename)
                    output_zipf.writestr(('/'.join(parts[1:]).replace(remove_folder_prefix, "")), file_data)

def uncompress_zip_to_folder(zip_path, extract_to):
    # Ensure the directory for extraction exists
    os.makedirs(extract_to, exist_ok=True)
    
    # Uncompress all files from the zip into the specified folder
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    # Delete the original zip file
    os.remove(zip_path)

repo_owner = "slavatrofimov"
repo_name = "amisandbox"
branch = "main"
folder_prefix = ""

download_folder_as_zip(repo_owner, repo_name, output_zip = "./builtin/workspace/workspace.zip", branch = branch, folder_to_extract= "/workspace", remove_folder_prefix = folder_prefix)
download_folder_as_zip(repo_owner, repo_name, output_zip = "./builtin/config/config.zip", branch = branch, folder_to_extract= "/config" , remove_folder_prefix = folder_prefix)

uncompress_zip_to_folder(zip_path = "./builtin/config/config.zip", extract_to= "./builtin")
#uncompress_zip_to_folder(zip_path = "./builtin/workspace/workspace.zip", extract_to= "./builtin")

In [4]:
base_path = './builtin/'

deploy_order_path = os.path.join(base_path, 'config/deployment_sequence.json')
with open(deploy_order_path, 'r') as file:
        deployment_order = json.load(file)


deployment_order = [
        {
        "name": "ReferenceDataLH.Lakehouse",
        "folder": "Store and Query",
        "id": "08d5aa3f-8374-9b1f-4a32-cbc7374bb522"
        },
        {
        "name": "AMI Telemetry and Outage Simulation.Notebook",
        "folder": "Simulation",
        "id": "735054a9-b72c-a2c0-4aa2-197db7d80dd8"
        },


        ]

mapping_table=[]

## Definition of deployment functions

In [ ]:
# Set environment parameters for Fabric CLI
token = notebookutils.credentials.getToken('pbi')
os.environ['FAB_TOKEN'] = token
os.environ['FAB_TOKEN_ONELAKE'] = token

def run_fab_command( command, capture_output: bool = False, silently_continue: bool = False):
    result = subprocess.run(["fab", "-c", command], capture_output=capture_output, text=True)
    if (not(silently_continue) and (result.returncode > 0 or result.stderr)):
       raise Exception(f"Error running fab command. exit_code: '{result.returncode}'; stderr: '{result.stderr}'")    
    if (capture_output): 
        output = result.stdout.strip()
        return output

def fab_get_id(name):
    id = run_fab_command(f"get /{trg_workspace_name}.Workspace/{name} -q id" , capture_output = True, silently_continue= True)
    return(id)

def get_id_by_name(name):
    for it in deployment_order:
        if it.get("name") == name:
                return it.get("id")
    return None

def copy_to_tmp(name, folder):
    shutil.rmtree("./builtin/tmp",  ignore_errors=True)
    path2zip = "./builtin/workspace/workspace.zip"
    with  ZipFile(path2zip) as archive:
        for file in archive.namelist():
            if file.startswith(f'workspace/{folder}{name}/'):    
                archive.extract(file, './builtin/tmp')
    return(f"./builtin/tmp/workspace/{folder}{name}")

def deploy_kql_database(eventhouse_name, kql_db_path, kql_db_name, parent_eventhouse_id):
    """
    Deploy a KQL Database as a child item of an Eventhouse using Fabric API.
    
    Args:
        eventhouse_name: Name of the parent Eventhouse
        kql_db_path: Path to the KQL database folder containing DatabaseProperties.json and DatabaseSchema.kql
        kql_db_name: Name of the KQL database to create
        parent_eventhouse_id: ID of the parent Eventhouse
    
    Returns:
        The ID of the created KQL database
    """
    try:
        # Read DatabaseProperties.json to get configuration
        db_properties_path = os.path.join(kql_db_path, 'DatabaseProperties.json')
        if os.path.exists(db_properties_path):
            with open(db_properties_path, 'r', encoding='utf-8') as file:
                db_properties = json.load(file)
        else:
            # Default properties if file doesn't exist
            db_properties = {
                "databaseType": "ReadWrite",
                "oneLakeCachingPeriod": "P36500D",
                "oneLakeStandardStoragePeriod": "P36500D"
            }
        
        # Update parent Eventhouse ID
        db_properties["parentEventhouseItemId"] = parent_eventhouse_id
        
        # Create KQL Database using Fabric API
        create_payload = {
            "displayName": kql_db_name,
            "type": "KQLDatabase",
            "description": f"KQL Database {kql_db_name}",
            "definition": {
                "parts": [
                    {
                        "path": "DatabaseProperties.json",
                        "payload": json.dumps(db_properties, indent=2),
                        "payloadType": "InlineBase64"
                    }
                ]
            }
        }
        
        # Convert payload to JSON string for the API call
        create_payload_json = json.dumps(create_payload).replace('"', '\\"')
        
        print(f"Creating KQL Database: {kql_db_name}")
        
        # Create the KQL database using Fabric API
        create_result = run_fab_command(
            f'api -A fabric -X post workspaces/{trg_workspace_id}/items -i "{create_payload_json}"',
            capture_output=True,
            silently_continue=True
        )
        
        if create_result:
            result_json = json.loads(create_result)
            kql_db_id = result_json.get('text', {}).get('id')
            print(f"Created KQL Database with ID: {kql_db_id}")
        else:
            # Try to get the ID if it already exists
            kql_db_id = fab_get_id(f"{eventhouse_name}/{kql_db_name}")
            print(f"KQL Database may already exist, retrieved ID: {kql_db_id}")
        
        # Deploy schema if DatabaseSchema.kql exists
        schema_path = os.path.join(kql_db_path, 'DatabaseSchema.kql')
        if os.path.exists(schema_path):
            print(f"Deploying schema from: {schema_path}")
            
            with open(schema_path, 'r', encoding='utf-8') as file:
                schema_content = file.read()
            
            # Execute the KQL schema commands
            if schema_content.strip():
                # Split schema into individual commands and execute them
                kql_commands = [cmd.strip() for cmd in schema_content.split('\n') if cmd.strip() and not cmd.strip().startswith('//')]
                
                for i, command in enumerate(kql_commands):
                    if command:
                        try:
                            print(f"Executing KQL command {i+1}/{len(kql_commands)}")
                            
                            # Prepare the KQL command payload
                            kql_payload = {
                                "csl": command,
                                "db": kql_db_name
                            }
                            
                            kql_payload_json = json.dumps(kql_payload).replace('"', '\\"')
                            
                            # Execute KQL command using Fabric API
                            run_fab_command(
                                f'api -A fabric -X post workspaces/{trg_workspace_id}/items/{kql_db_id}/runKqlCommand -i "{kql_payload_json}"',
                                capture_output=False,
                                silently_continue=True
                            )
                            
                        except Exception as e:
                            print(f"Warning: Failed to execute KQL command {i+1}: {str(e)}")
                            continue
        
        return kql_db_id
        
    except Exception as e:
        print(f"Error deploying KQL Database {kql_db_name}: {str(e)}")
        return None

def replace_ids_in_folder(folder_path, mapping_table):
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith(('.py', '.json', '.pbir', '.platform', '.ipynb', '.tmdl')) and not file_name.endswith('report.json'):
                file_path = os.path.join(root, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    for mapping in mapping_table:  
                        content = content.replace(mapping["old_id"], mapping["new_id"])
                with open(file_path, 'w', encoding='utf-8') as file:
                    file.write(content)

def replace_githubRef_in_folder(folder_path, repo_owner, repo_name, branch):
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith(('.py', '.json', '.pbir', '.platform', '.ipynb')) and not file_name.endswith('report.json'):
                file_path = os.path.join(root, file_name)
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    matches = re.findall(r'https://raw\.githubusercontent\.com/([^/]+/[^/]+/[^/]+/[^/]+/[^/]+)/(.*)', content, re.MULTILINE)
                    if matches:
                        for match in matches:
                            oldUrl = f"https://raw.githubusercontent.com/{match[0]}/{match[1]}"
                            newUrl = f"https://raw.githubusercontent.com/{repo_owner}/{repo_name}/refs/heads/{branch}/{match[1]}"
                            content = content.replace(oldUrl, newUrl)
                        with open(file_path, 'w', encoding='utf-8') as file:
                                file.write(content)

def get_semantic_model_id(report_folder):
    definition_file = os.path.join(report_folder, 'definition.pbir')
    if os.path.exists(definition_file):
        with open(definition_file, 'r', encoding='utf-8') as file:
            content = json.load(file)
            semantic_model_id = content.get('datasetReference', {}).get('byConnection', {}).get('pbiModelDatabaseName')
            if semantic_model_id:
                return semantic_model_id
    return None

def update_sm_connection_to_ami_lakehouse(semantic_model_folder):
    new_sm_db= run_fab_command(f"get /{trg_workspace_name}.Workspace/AMI.Lakehouse -q properties.sqlEndpointProperties.connectionString", capture_output = True, silently_continue=True)
    new_lakehouse_sql_id= run_fab_command(f"get /{trg_workspace_name}.Workspace/AMI.Lakehouse -q properties.sqlEndpointProperties.id", capture_output = True, silently_continue=True)
        
    expressions_file = os.path.join(semantic_model_folder, 'definition', 'expressions.tmdl')
    if os.path.exists(expressions_file):
        with open(expressions_file, 'r', encoding='utf-8') as file:
            content = file.read()
            match = re.search(r'Sql\.Database\("([^"]+)",\s*"([^"]+)"\)', content)
            if match:
                old_sm_db, old_lakehouse_sql_id = match.group(1), match.group(2)
                content = content.replace(old_sm_db, new_sm_db).replace(old_lakehouse_sql_id, new_lakehouse_sql_id)
                with open(expressions_file, 'w', encoding='utf-8') as file:
                    file.write(content)

def update_report_definition( path): 
    semantic_model_id = get_semantic_model_id(path)
    definition_path = os.path.join(path, "definition.pbir")
   
    with open(definition_path, "r", encoding="utf8") as file:
        report_definition = json.load(file)

    report_definition["datasetReference"]["byPath"] = None

    by_connection_obj = {
            "connectionString": None,
            "pbiServiceModelId": None,
            "pbiModelVirtualServerName": "sobe_wowvirtualserver",
            "pbiModelDatabaseName": semantic_model_id,
            "name": "EntityDataSource",
            "connectionType": "pbiServiceXmlaStyleLive",
        }

    report_definition["datasetReference"]["byConnection"] = by_connection_obj

    with open(definition_path, "w") as file:
            json.dump(report_definition, file, indent=4)

def print_color(text, state):
    red  = '\033[91m'
    yellow = '\033[93m'  
    green = '\033[92m'   
    white = '\033[0m'  
    if state == "error":
        print(red, text, white)
    elif state == "warning":
        print(yellow, text, white)
    elif state == "success":
        print(green, text, white)
    else:
        print("", text)

## Get current Workspace
This cell gets the current workspace to deploy AMI automatically inside it

In [6]:
trg_workspace_id = fabric.get_notebook_workspace_id()
res = run_fab_command(f"api -X get workspaces/{trg_workspace_id}" , capture_output = True, silently_continue=True)
trg_workspace_name = json.loads(res)["text"]["displayName"]

print(f"Current workspace: {trg_workspace_name}")
print(f"Current workspace ID: {trg_workspace_id}")

mapping_table.append({ "old_id": get_id_by_name("Focus"), "new_id": trg_workspace_id })
mapping_table.append({ "old_id": "00000000-0000-0000-0000-000000000000", "new_id": trg_workspace_id })

Current workspace: ST-AMI Sandbox
Current workspace ID: 57347e55-c195-4e9b-96b8-03c4703f23c2


## Deployment Logic
This part iterates through all the items, gets the respective source code, replaces all IDs dynamically and deploys the new item

In [7]:
tmp_path = copy_to_tmp("AMI Telemetry and Outage Simulation.Notebook", "Simulation/")
print(tmp_path)

./builtin/tmp/workspace/Simulation/AMI Telemetry and Outage Simulation.Notebook


In [ ]:
exclude = ["Focus", 'Deploy_AMI']

for it in deployment_order:

    new_id = None
    
    name = it["name"]
    folder = "" if it["folder"] == "" else it["folder"] + "/"

    print(folder)
    print(name)


    if name in exclude:
            continue

    print("")
    print("#############################################")
    print(f"Deploying {folder}{name}")

    # Copy and replace IDs in the item
    tmp_path = copy_to_tmp(name, folder)
    print(tmp_path)
    replace_ids_in_folder(tmp_path, mapping_table)
    replace_githubRef_in_folder(tmp_path,repo_owner,repo_name,branch)

    cli_parameter = ''
    if ".Notebook" in name:
        cli_parameter = cli_parameter + " --format .ipynb"
    elif ".Lakehouse" in name:
        run_fab_command(f"create /{trg_workspace_name}.Workspace/{name}" , silently_continue=True )
        new_id = fab_get_id(name)
        mapping_table.append({ "old_id": get_id_by_name(name), "new_id": new_id })
        
        continue
    elif ".Eventhouse" in name:
        # Deploy Eventhouse first (this creates the container)
        run_fab_command(f"import /{trg_workspace_name}.Workspace/{name} -i {tmp_path}", silently_continue=True)
        new_id = fab_get_id(name)
        mapping_table.append({ "old_id": it["id"], "new_id": new_id })
        
        # Check for KQL Database children and deploy them
        children_path = os.path.join(tmp_path, '.children')
        if os.path.exists(children_path):
            for child_item in os.listdir(children_path):
                if child_item.endswith('.KQLDatabase'):
                    kql_db_path = os.path.join(children_path, child_item)
                    kql_db_name = child_item.replace('.KQLDatabase', '')
                    
                    print(f"Deploying KQL Database: {kql_db_name}")
                    kql_db_id = deploy_kql_database(name.replace('.Eventhouse', ''), kql_db_path, kql_db_name, new_id)
                    
                    if kql_db_id:
                        # Add mapping for the KQL database if we have its original ID
                        kql_platform_file = os.path.join(kql_db_path, '.platform')
                        if os.path.exists(kql_platform_file):
                            with open(kql_platform_file, 'r', encoding='utf-8') as file:
                                platform_data = json.load(file)
                                old_kql_id = platform_data.get('config', {}).get('logicalId')
                                if old_kql_id:
                                    mapping_table.append({ "old_id": old_kql_id, "new_id": kql_db_id })
        
        continue
    elif ".Report" in name:
        update_report_definition(  tmp_path  )
    elif ".SemanticModel" in name:
        update_sm_connection_to_ami_lakehouse(tmp_path)
    
    run_fab_command(f"import  /{trg_workspace_name}.Workspace/{name} -i {tmp_path} -f {cli_parameter} ", silently_continue= True)
    new_id= fab_get_id(name)
    mapping_table.append({ "old_id": it["id"], "new_id": new_id })

Store and Query/
ReferenceDataLH.Lakehouse

#############################################
Deploying Store and Query/ReferenceDataLH.Lakehouse
./builtin/tmp/workspace/Store and Query/ReferenceDataLH.Lakehouse


TypeError: replace() argument 1 must be str, not None

## Post-Deployment logic
In this separate notebook, all needed tables for AMI are automatically deployed.

In [ ]:
# Fill default tables
time.sleep(10)
run_fab_command('job run ' + trg_workspace_name + '.Workspace/Init_AMI_Lakehouse_Tables.Notebook -i {"parameters": {"_inlineInstallationEnabled": {"type": "Bool", "value": "True"} } }')

In [ ]:
# Refresh SQL Endpoint for Config_Lakehouse
items = run_fab_command(f'api -X get -A fabric /workspaces/{trg_workspace_id}/items' , capture_output = True)
for it in json.loads(items)['text']['value']:
    if (it['displayName'] == 'AMI' ) & (it['type'] =='SQLEndpoint' ):
        lh_sql_endpoint = it['id']
print(f"AMI Lakehouse SQL Endpoint ID: {lh_sql_endpoint}")

try:
    run_fab_command(f'api -A fabric -X post workspaces/{trg_workspace_id}/sqlEndpoints/{lh_sql_endpoint}/refreshMetadata?preview=True -i {{}} ', capture_output=True)
    print("Refresh AMI_SQL_Endpoint")
except:
    print("SQL Endpoint Refresh API failed, it is still in Preview, so there can be changes")

## KQL Database Deployment Explained

The `deploy_kql_database()` function handles the deployment of KQL Databases as child items of Eventhouses using the following approach:

### Why Use Fabric API Instead of Import?

- **Fabric CLI Limitation**: While `fab create` can create a KQL database structure, `fab import` cannot properly import KQL database content including schema, tables, and functions
- **API Solution**: The `fab api` command allows us to send direct REST API calls to the Fabric service for more complete control

### Deployment Process:

1. **Read Configuration**: Loads `DatabaseProperties.json` to get database settings (type, caching periods, etc.)
2. **Update Parent Reference**: Sets the `parentEventhouseItemId` to link the KQL database to its Eventhouse
3. **Create Database**: Uses Fabric API POST call to create the KQL database structure
4. **Deploy Schema**: Reads `DatabaseSchema.kql` and executes each KQL command individually using the `runKqlCommand` API
5. **ID Mapping**: Tracks old/new IDs for cross-reference replacement

### Key Features:

- **Handles Complex KQL**: Supports tables, functions, materialized views, update policies, and encoding policies
- **Error Resilience**: Continues deployment even if individual KQL commands fail (useful for dependencies)
- **Parent-Child Relationship**: Maintains proper Eventhouse ↔ KQL Database relationships
- **ID Replacement**: Ensures all cross-references use new deployment IDs

### Usage in Deployment:

The function is automatically called when deploying `.Eventhouse` items that contain `.children/*.KQLDatabase` folders.

In [ ]:
string = '''{
"databaseType": "ReadWrite",
"parentEventhouseItemId": "0ebce58b-207c-4d5c-b4e4-a9da0dc9806c",
"oneLakeCachingPeriod": "P36500D",
"oneLakeStandardStoragePeriod": "P36500D"
}'''